In [1]:
# region Import
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import datetime as dt

sns.set()
pd.set_option('display.expand_frame_repr', False)
# endregion

In [5]:
pd.__version__

'0.25.1'

In [3]:
filePickle = 'C:/Users/phan/OneDrive - adesso Group/DataSet/Online Retail.pickle'
with open(file=filePickle, mode='rb') as f:
   df = pickle.load(f)

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df = df[df['Country'] == 'United Kingdom']
df = df[df['Quantity'] > 0]
df = df[['CustomerID', 'InvoiceDate', 'InvoiceNo', 'Quantity', 'UnitPrice']]
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
PRESENT = dt.datetime(2011, 12, 10)

In [8]:
rfm: pd.DataFrame = df.groupby('CustomerID').agg({
   'InvoiceDate': lambda date: (PRESENT - date.max()).days,
   'InvoiceNo': lambda num: len(num),
   'TotalPrice': lambda price: price.sum()
})

In [9]:
rfm.columns = ['recency', 'frequency', 'monetary']
qua = pd.qcut(x=range(100), q=4, labels=['1', '2', '3', '4'])

In [10]:
rfm['rQuartile'] = pd.qcut(x=rfm['recency'], q=4, labels=['1', '2', '3', '4'])
rfm['fQuartile'] = pd.qcut(x=rfm['frequency'], q=4, labels=['4', '3', '2', '1'])
rfm['mQuartile'] = pd.qcut(x=rfm['monetary'], q=4, labels=['4', '3', '2', '1'])

In [12]:
rfm['rfmScore'] = rfm['rQuartile'].astype('str') + rfm['fQuartile'].astype('str') + rfm['mQuartile'].astype('str')

In [15]:
rfm.sort_values('rfmScore').head()

,recency,frequency,monetary,rQuartile,fQuartile,mQuartile,rfmScore
CustomerID,,,,,,,
16814.0,14,324,6509.10,1,1,1,111
15214.0,1,110,1661.44,1,1,1,111
17750.0,5,193,1797.57,1,1,1,111
17744.0,3,136,2322.40,1,1,1,111
16839.0,8,266,16594.35,1,1,1,111


In [20]:
df.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice
0,17850.0,2010-12-01 08:26:00,536365,6,2.55,15.30
1,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34
2,17850.0,2010-12-01 08:26:00,536365,8,2.75,22.00
3,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34
4,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486286 entries, 0 to 541893
Data columns (total 6 columns):
CustomerID     354345 non-null float64
InvoiceDate    486286 non-null datetime64[ns]
InvoiceNo      486286 non-null object
Quantity       486286 non-null int64
UnitPrice      486286 non-null float64
TotalPrice     486286 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 26.0+ MB


In [25]:
df['CustomerID'].value_counts().head()

17841.0    7847
14096.0    5111
12748.0    4596
14606.0    2700
15311.0    2379
Name: CustomerID, dtype: int64

In [27]:
df[df['CustomerID'] == 13081.0].head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice
7314,13081.0,2010-12-03 17:20:00,537034,96,1.85,177.60
7315,13081.0,2010-12-03 17:20:00,537034,96,1.85,177.60
7316,13081.0,2010-12-03 17:20:00,537034,96,1.85,177.60
7317,13081.0,2010-12-03 17:20:00,537034,96,1.85,177.60
7318,13081.0,2010-12-03 17:20:00,537034,48,0.29,13.92


In [31]:
test = df.groupby(['CustomerID', 'InvoiceNo']).agg({
    'TotalPrice': lambda prices: sum(prices),
    'InvoiceDate': lambda date: min(date)
})

In [33]:
test.head(10)

TotalPrice         InvoiceDate
CustomerID InvoiceNo                                
12346.0    541431       77183.60 2011-01-18 10:01:00
12747.0    537215         358.56 2010-12-05 15:38:00
           538537         347.71 2010-12-13 10:41:00
           541677         303.04 2011-01-20 14:01:00
           545321         310.78 2011-03-01 14:53:00
           551992         442.96 2011-05-05 15:31:00
           554549         328.35 2011-05-25 09:57:00
           558265         376.30 2011-06-28 10:06:00
           563949         301.70 2011-08-22 10:38:00
           569397         675.38 2011-10-04 08:26:00

In [39]:
df[(df['CustomerID'] == 12747.0) & (df['InvoiceNo'] == 537215)]

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice
9754,12747.0,2010-12-05 15:38:00,537215,12,2.55,30.60
9755,12747.0,2010-12-05 15:38:00,537215,6,2.55,15.30
9756,12747.0,2010-12-05 15:38:00,537215,16,1.69,27.04
9757,12747.0,2010-12-05 15:38:00,537215,24,1.65,39.60
9758,12747.0,2010-12-05 15:38:00,537215,6,5.45,32.70
9759,12747.0,2010-12-05 15:38:00,537215,36,5.55,199.80
9760,12747.0,2010-12-05 15:38:00,537215,8,1.69,13.52


In [98]:
df['dayofyear'] = df['InvoiceDate'].dt.dayofyear
df[df['CustomerID'] == 12747.0]

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice,dayofyear
9754,12747.0,2010-12-05 15:38:00,537215,12,2.55,30.60,339
9755,12747.0,2010-12-05 15:38:00,537215,6,2.55,15.30,339
9756,12747.0,2010-12-05 15:38:00,537215,16,1.69,27.04,339
9757,12747.0,2010-12-05 15:38:00,537215,24,1.65,39.60,339
9758,12747.0,2010-12-05 15:38:00,537215,6,5.45,32.70,339
9759,12747.0,2010-12-05 15:38:00,537215,36,5.55,199.80,339
9760,12747.0,2010-12-05 15:38:00,537215,8,1.69,13.52,339
27390,12747.0,2010-12-13 10:41:00,538537,16,5.95,95.20,347
27391,12747.0,2010-12-13 10:41:00,538537,2,7.95,15.90,347
27392,12747.0,2010-12-13 10:41:00,538537,24,2.55,61.20,347


In [91]:
dupRemoved = df.drop_duplicates(subset=['CustomerID', 'dayofyear'])
dupRemoved.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice,dayofyear
0,17850.0,2010-12-01 08:26:00,536365,6,2.55,15.30,335
9,13047.0,2010-12-01 08:34:00,536367,32,1.69,54.08,335
46,13748.0,2010-12-01 09:00:00,536371,80,2.55,204.00,335
65,15100.0,2010-12-01 09:09:00,536374,32,10.95,350.40,335
82,15291.0,2010-12-01 09:32:00,536376,48,3.45,165.60,335


In [92]:
diff = dupRemoved[dupRemoved['CustomerID'] == 17850.0].drop(['Quantity', 'UnitPrice', 'TotalPrice'], axis=1)
diff

,CustomerID,InvoiceDate,InvoiceNo,dayofyear
0,17850.0,2010-12-01 08:26:00,536365,335
3118,17850.0,2010-12-02 08:32:00,536600,336


In [80]:
dupRemoved['InvoiceDate'].dt.dayofyear

0         335
7         335
9         335
21        335
25        335
46        335
47        335
49        335
65        335
66        335
82        335
84        335
86        335
105       335
106       335
142       335
155       335
168       335
175       335
178       335
183       335
211       335
242       335
252       335
253       335
264       335
278       335
296       335
298       335
315       335
         ... 
540415    343
540421    343
540423    343
540426    343
540442    343
540443    343
540446    343
540451    343
541217    343
541232    343
541264    343
541323    343
541542    343
541545    343
541569    343
541589    343
541613    343
541628    343
541702    343
541703    343
541731    343
541751    343
541806    343
541836    343
541860    343
541863    343
541865    343
541867    343
541869    343
541890    343
Name: InvoiceDate, Length: 18825, dtype: int64

In [62]:
df[df['CustomerID'] == 17850.0].groupby('InvoiceNo').agg({
    'TotalPrice': lambda price: sum(price)
}).head()


,TotalPrice
InvoiceNo,
536365,139.12
536366,22.20
536372,22.20
536373,259.86
536375,259.86
